In [1]:
import datetime
from pyspark.sql import DataFrame, Row
from pyspark.sql.types import *
from pyspark.sql.functions import unix_timestamp, from_unixtime, col
from pyspark.sql import functions as F
import sys
from hops import hdfs as hdfs
print(hdfs.project_path())

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
3,application_1614293057610_0004,pyspark,idle,Link,Link


SparkSession available as 'spark'.
hdfs://rpc.namenode.service.consul:8020/Projects/test/

In [9]:
fg1_schema = StructType([
  StructField("id", IntegerType(), True),
  StructField("ts", IntegerType(), True),
  StructField("f1", StringType(), True)    
])

fg1=spark.read.csv("hdfs:///Projects/" + hdfs.project_name() + "/Resources/10000000-20-1-out.csv", header=True, schema=fg1_schema)
fg1=fg1.sort(col("id"),col("ts"))
fg1.show()

+---+----+---+
| id|  ts| f1|
+---+----+---+
|  1|1020| f1|
|  1|1040| f1|
|  1|1060| f1|
|  1|1080| f1|
|  1|1100| f1|
|  1|1120| f1|
|  1|1140| f1|
|  1|1160| f1|
|  1|1180| f1|
|  1|1200| f1|
|  1|1220| f1|
|  1|1240| f1|
|  1|1260| f1|
|  1|1280| f1|
|  1|1300| f1|
|  1|1320| f1|
|  1|1340| f1|
|  1|1360| f1|
|  1|1380| f1|
|  1|1400| f1|
+---+----+---+
only showing top 20 rows

In [10]:
fg2_schema = StructType([
  StructField("id", IntegerType(), True),
  StructField("ts", IntegerType(), True),
  StructField("f2", StringType(), True)    
])

fg2=spark.read.csv("hdfs:///Projects/" + hdfs.project_name() + "/Resources/10000000-20-1-out.csv", header=True, schema=fg2_schema)
fg2=fg2.select([col("id").alias("id_2"), col("ts").alias("ts_2"), col("f2")])
fg2.show()

+----+----+---+
|id_2|ts_2| f2|
+----+----+---+
|   1|1020| f1|
|   1|1040| f1|
|   1|1060| f1|
|   1|1080| f1|
|   1|1100| f1|
|   1|1120| f1|
|   1|1140| f1|
|   1|1160| f1|
|   1|1180| f1|
|   1|1200| f1|
|   1|1220| f1|
|   1|1240| f1|
|   1|1260| f1|
|   1|1280| f1|
|   1|1300| f1|
|   1|1320| f1|
|   1|1340| f1|
|   1|1360| f1|
|   1|1380| f1|
|   1|1400| f1|
+----+----+---+
only showing top 20 rows

In [11]:
from pyspark.sql import Window
from pyspark.sql.functions import lit, when, col, lag, rank

win = Window.partitionBy(["id", "ts"]).orderBy(col('ts_2').desc())

In [12]:
final=fg1.join(fg2, (fg1.id == fg2.id_2) & (fg1.ts >= fg2.ts_2)) \
    .withColumn("id_rank", rank().over(win)) \
    .filter(col("id_rank") == 1).drop(col("id_rank")).drop(col("ts_2")).drop(col("id_2"))

In [13]:
final.show()

+---+----+---+---+
| id|  ts| f1| f2|
+---+----+---+---+
|148|1020| f1| f1|
|148|1040| f1| f1|
|148|1060| f1| f1|
|148|1080| f1| f1|
|148|1100| f1| f1|
|148|1120| f1| f1|
|148|1140| f1| f1|
|148|1160| f1| f1|
|148|1180| f1| f1|
|148|1200| f1| f1|
|148|1220| f1| f1|
|148|1240| f1| f1|
|148|1260| f1| f1|
|148|1280| f1| f1|
|148|1300| f1| f1|
|148|1320| f1| f1|
|148|1340| f1| f1|
|148|1360| f1| f1|
|148|1380| f1| f1|
|148|1400| f1| f1|
+---+----+---+---+
only showing top 20 rows

In [ ]:
final.write.parquet(hdfs.project_path() + "Resources/joined.parquet")